In [151]:
from pyspark.sql import SparkSession
import findspark

findspark.init()
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()


In [17]:
import os
import pymongo
import pandas as pd
from dotenv import load_dotenv
load_dotenv('../../config/.env')
mongo_uri = os.getenv('MONGO_URI')
client = pymongo.MongoClient(f'{mongo_uri}')
db = client['name']
collection = db['india_name']


document = list(collection.find({},{ "_id": 0,}))
df = pd.DataFrame(document)

collection = db['us_name']
document = list(collection.find({},{ "_id": 0,}))
df= pd.concat([df, pd.DataFrame(document)])

collection = db['thai_name']
document = list(collection.find({},{ "_id": 0,}))
df= pd.concat([df, pd.DataFrame(document)])

collection = db['china_name']
document = list(collection.find({},{ "_id": 0,}))
df= pd.concat([df, pd.DataFrame(document)])

client.close()

df.shape



(166850, 2)

ฺื**NOT USED**

In [12]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

df = pd.DataFrame(document)
df['gender'] = df['gender'].map({'f': 0, 'm': 1})

cv = CountVectorizer()
X = cv.fit_transform(df['name'].values.astype('U'))
cv.get_feature_names_out()

y = df['gender']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(X_train, y_train)
nb.score(X_test, y_test)




0.5785804816223067

Use this one, added Feature on Naive Bayes, will add pipeline and pickle later

In [ ]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier

def features(name):
    name = str(name)
    name = name.lower()
    return {
        'first-letter': name[0],
        'last-letter': name[-1],
        'first2-letter': name[:2],
        'last2-letter': name[-2:],
        'first3-letter': name[:3],
        'last3-letter': name[-3:],
        
    }
    
features = np.vectorize(features)

df_X = features(df['name'])
df_y = df['gender']



# dv.fit(names_sample)
# transformed = dv.transform(names_sample)
# transformed.toarray()

# dv.get_feature_names_out()

X_train, X_test, y_train, y_test = train_test_split(df_X, df_y, test_size=0.3, random_state=2024)

dv = DictVectorizer()
dv.fit_transform(X_train)


rf = RandomForestClassifier()
rf.fit(dv.transform(X_train), y_train)
rf.score(dv.transform(X_test), y_test)



In [ ]:
sample = ['Nithi', 'Rawit', 'Thamvarut', 'Pasit', 'Peeradon', 'Kittiphop', 'Nigga', 'Aranya', 'Kamon', 'Dao',]
transform = dv.transform(features(sample)).toarray()
rf.predict(transform)
report = classification_report(y_test, rf.predict(dv.transform(X_test)))

for pickle

In [ ]:
import pickle
from sklearn.pipeline import FunctionTransformer, Pipeline


# df = pd.DataFrame(document)
df = df[~df['name'].duplicated(keep=False)]

# print(df['name'].value_counts().sort_values())

def features(name):
    name = str(name).lower()
    return {
        'first-letter': name[0],
        'last-letter': name[-1],
        'first2-letter': name[:2],
        'last2-letter': name[-2:],
        'first3-letter': name[:3],
        'last3-letter': name[-3:],
        'first4-letter': name[:4],
        'last4-letter': name[-4:]
    }
    
X = df['name']
y = df['gender']
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)

pipeline = Pipeline([
    ('feature_extractor', FunctionTransformer(np.vectorize(features), validate=False)),
    ('vectorizer', DictVectorizer()),
    ('classifier', MultinomialNB())
])

pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)
print(classification_report(y_test, y_pred))


names = ['rawit', 'nithi', 'thamvarut', 'pasit', 'peeradon', 'kittiphop']

report = pipeline.predict(names)
print(report)

with open('name_pipeline.pkl', 'wb') as f:
    pickle.dump(pipeline, f)

              precision    recall  f1-score   support

           f       0.90      0.89      0.89     16170
           m       0.82      0.84      0.83      9603

    accuracy                           0.87     25773
   macro avg       0.86      0.86      0.86     25773
weighted avg       0.87      0.87      0.87     25773

['m' 'f' 'm' 'm' 'm' 'm']


In [ ]:
with open('name_pipeline.pkl', 'rb') as f:
    load_pipe = pickle.load(f)
    
names = ['rawit', 'nithi', 'thamvarut', 'pasit', 'peeradon', 'kittiphop']

report = load_pipe.predict(names)
print(report)

['m' 'f' 'm' 'm' 'm' 'm']


In [207]:
df.shape

(128862, 2)